In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('example.db')


def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)


In [5]:
# df = pd.read_csv('./IncidentRollupTable.csv')
# df.to_sql('IncidentRollup', conn, if_exists='replace', index=False)

770

In [6]:
rsq("""-- Fill in the appropriate aggregate functions
SELECT
	it.IncidentType,
	COUNT(1) AS NumberOfRows,
	SUM(ir.NumberOfIncidents) AS TotalNumberOfIncidents,
	MIN(ir.NumberOfIncidents) AS MinNumberOfIncidents,
	MAX(ir.NumberOfIncidents) AS MaxNumberOfIncidents,
	MIN(ir.IncidentDate) As MinIncidentDate,
	MAX(ir.IncidentDate) AS MaxIncidentDate
FROM IncidentRollup ir
	INNER JOIN IncidentType it
		ON ir.IncidentTypeID = it.IncidentTypeID
WHERE
	ir.IncidentDate BETWEEN '2019-08-01' AND '2019-10-31'
GROUP BY
	it.IncidentType;""")

,IncidentType,NumberOfRows,TotalNumberOfIncidents,MinNumberOfIncidents,MaxNumberOfIncidents,MinIncidentDate,MaxIncidentDate
0,Clicked malicious link,24,110,1,12,2019-08-01,2019-10-28
1,Forgot key card,52,216,1,10,2019-08-01,2019-10-31
2,Installed unapproved software,53,251,1,10,2019-08-01,2019-10-29
3,Left computer unlocked,70,366,1,11,2019-08-01,2019-10-31
4,Ransomware installed,12,79,3,11,2019-08-01,2019-10-23


In [8]:
rsq("""-- Fill in the functions and columns
SELECT
	COUNT(DISTINCT ir.IncidentTypeID) AS NumberOfIncidentTypes,
	COUNT(DISTINCT ir.IncidentDate) AS NumberOfDaysWithIncidents
FROM IncidentRollup ir
WHERE
ir.IncidentDate BETWEEN '2019-08-01' AND '2019-10-31';


""")

,NumberOfIncidentTypes,NumberOfDaysWithIncidents
0,5,91


In [9]:
rsq("""SELECT
	it.IncidentType,
    -- Fill in the appropriate expression
	SUM(CASE WHEN ir.NumberOfIncidents > 5 THEN 1 ELSE 0 END) AS NumberOfBigIncidentDays,
    -- Number of incidents will always be at least 1, so
    -- no need to check the minimum value, just that it's
    -- less than or equal to 5
    SUM(CASE WHEN ir.NumberOfIncidents <= 5 THEN 1 ELSE 0 END) AS NumberOfSmallIncidentDays
FROM IncidentRollup ir
	INNER JOIN IncidentType it
		ON ir.IncidentTypeID = it.IncidentTypeID
WHERE
	ir.IncidentDate BETWEEN '2019-08-01' AND '2019-10-31'
GROUP BY
it.IncidentType;""")

,IncidentType,NumberOfBigIncidentDays,NumberOfSmallIncidentDays
0,Clicked malicious link,8,16
1,Forgot key card,15,37
2,Installed unapproved software,18,35
3,Left computer unlocked,32,38
4,Ransomware installed,9,3


In [12]:
rsq("""SELECT
    it.IncidentType,
    AVG(ir.NumberOfIncidents) AS MeanNumberOfIncidents,
    AVG(CAST(ir.NumberOfIncidents AS DECIMAL(4,2))) AS MeanNumberOfIncidents,
    SQRT(AVG(ir.NumberOfIncidents * ir.NumberOfIncidents) - AVG(ir.NumberOfIncidents) * AVG(ir.NumberOfIncidents)) AS NumberOfIncidentsStandardDeviation,
    AVG(ir.NumberOfIncidents * ir.NumberOfIncidents) - AVG(ir.NumberOfIncidents) * AVG(ir.NumberOfIncidents) AS NumberOfIncidentsVariance,
    COUNT(1) AS NumberOfRows
FROM IncidentRollup ir
    INNER JOIN IncidentType it
        ON ir.IncidentTypeID = it.IncidentTypeID
	INNER JOIN Calendar c
        INNER JOIN Calendar c
        ON ir.IncidentDate = c.Date
WHERE
    c.CalendarQuarter = 2
    AND c.CalendarYear = 2020
GROUP BY
it.IncidentType;""")

DatabaseError: Execution failed on sql 'SELECT
    it.IncidentType,
    AVG(ir.NumberOfIncidents) AS MeanNumberOfIncidents,
    AVG(CAST(ir.NumberOfIncidents AS DECIMAL(4,2))) AS MeanNumberOfIncidents,
    SQRT(AVG(ir.NumberOfIncidents * ir.NumberOfIncidents) - AVG(ir.NumberOfIncidents) * AVG(ir.NumberOfIncidents)) AS NumberOfIncidentsStandardDeviation,
    AVG(ir.NumberOfIncidents * ir.NumberOfIncidents) - AVG(ir.NumberOfIncidents) * AVG(ir.NumberOfIncidents) AS NumberOfIncidentsVariance,
    COUNT(1) AS NumberOfRows
FROM IncidentRollup ir
    INNER JOIN IncidentType it
        ON ir.IncidentTypeID = it.IncidentTypeID
	INNER JOIN Calendar c
        INNER JOIN Calendar c
        ON ir.IncidentDate = c.Date
WHERE
    c.CalendarQuarter = 2
    AND c.CalendarYear = 2020
GROUP BY
it.IncidentType;': ambiguous column name: c.CalendarQuarter

In [15]:
# rsq("""SELECT
#     it.IncidentType,
#     AVG(ir.NumberOfIncidents) AS MeanNumberOfIncidents,
#     AVG(CAST(ir.NumberOfIncidents AS DECIMAL(4,2))) AS MeanNumberOfIncidents,
#     SQRT(AVG(ir.NumberOfIncidents * ir.NumberOfIncidents) - AVG(ir.NumberOfIncidents) * AVG(ir.NumberOfIncidents)) AS NumberOfIncidentsStandardDeviation,
#     AVG(ir.NumberOfIncidents * ir.NumberOfIncidents) - AVG(ir.NumberOfIncidents) * AVG(ir.NumberOfIncidents) AS NumberOfIncidentsVariance,
#     COUNT(1) AS NumberOfRows
# FROM IncidentRollup ir
#     INNER JOIN IncidentType it
#         ON ir.IncidentTypeID = it.IncidentTypeID
#     INNER JOIN Calendar c
#         ON ir.IncidentDate = c.Date
# WHERE
#     c.CalendarQuarter = 2
#     AND c.CalendarYear = 2020
# GROUP BY
# it.IncidentType;""")
rsq("""SELECT
    it.IncidentType,
    COUNT(1) AS NumberOfRows,
    SUM(ir.NumberOfIncidents) AS TotalNumberOfIncidents,
    MIN(ir.NumberOfIncidents) AS MinNumberOfIncidents,
    MAX(ir.NumberOfIncidents) AS MaxNumberOfIncidents,
    MIN(ir.IncidentDate) As MinIncidentDate,
    MAX(ir.IncidentDate) AS MaxIncidentDate
FROM IncidentRollup ir
    INNER JOIN IncidentType it
        ON ir.IncidentTypeID = it.IncidentTypeID
    INNER JOIN Calendar c
        ON ir.IncidentDate = c.Date
WHERE
    c.CalendarQuarter = 2
    AND c.CalendarYear = 2020
GROUP BY
it.IncidentType;""")

,IncidentType,NumberOfRows,TotalNumberOfIncidents,MinNumberOfIncidents,MaxNumberOfIncidents,MinIncidentDate,MaxIncidentDate
0,Clicked malicious link,21,98,1,10,2020-04-03,2020-06-28
1,Forgot key card,54,264,1,11,2020-04-02,2020-06-29
2,Installed unapproved software,31,177,1,12,2020-04-07,2020-06-26
3,Left computer unlocked,63,355,1,15,2020-04-01,2020-06-30
4,Ransomware installed,6,35,3,8,2020-04-10,2020-06-23


In [ ]:
SELECT DISTINCT
	it.IncidentType,
	AVG(CAST(ir.NumberOfIncidents AS DECIMAL(4,2)))
	    OVER(PARTITION BY it.IncidentType) AS MeanNumberOfIncidents,
    --- Fill in the missing value
	PERCENTILE_CONT(0.5)
    	-- Inside our group, order by number of incidents DESC
    	WITHIN GROUP (ORDER BY ir.NumberOfIncidents DESC)
        -- Do this for each IncidentType value
        OVER (PARTITION BY it.IncidentType) AS MedianNumberOfIncidents,
	COUNT(1) OVER (PARTITION BY it.IncidentType) AS NumberOfRows
FROM dbo.IncidentRollup ir
	INNER JOIN dbo.IncidentType it
		ON ir.IncidentTypeID = it.IncidentTypeID
	INNER JOIN dbo.Calendar c
		ON ir.IncidentDate = c.Date
WHERE
	c.CalendarQuarter = 2
	AND c.CalendarYear = 2020;

In [23]:
rsq("""
SELECT DISTINCT
    it.IncidentType,
    AVG(CAST(ir.NumberOfIncidents AS DECIMAL)) OVER(PARTITION BY it.IncidentType) AS MeanNumberOfIncidents,
    (SELECT AVG(x.NumberOfIncidents) 
     FROM (SELECT ir2.NumberOfIncidents 
           FROM IncidentRollup ir2 
           INNER JOIN IncidentType it2 ON ir2.IncidentTypeID = it2.IncidentTypeID 
           INNER JOIN Calendar c2 ON ir2.IncidentDate = c2.Date 
           WHERE it2.IncidentType = it.IncidentType AND c2.CalendarQuarter = 2 AND c2.CalendarYear = 2020 
           ORDER BY ir2.NumberOfIncidents 
           LIMIT 2 - (SELECT COUNT(*) 
                      FROM IncidentRollup ir3 
                      INNER JOIN IncidentType it3 ON ir3.IncidentTypeID = it3.IncidentTypeID 
                      INNER JOIN Calendar c3 ON ir3.IncidentDate = c3.Date 
                      WHERE it3.IncidentType = it.IncidentType AND c3.CalendarQuarter = 2 AND c3.CalendarYear = 2020 
                      AND ir3.NumberOfIncidents >= ir.NumberOfIncidents)) x) AS MedianNumberOfIncidents,
    COUNT(1) OVER (PARTITION BY it.IncidentTypeID) AS NumberOfRows
FROM IncidentRollup ir
    INNER JOIN IncidentType it ON ir.IncidentTypeID = it.IncidentTypeID
    INNER JOIN Calendar c ON ir.IncidentDate = c.Date
WHERE c.CalendarQuarter = 2
    AND c.CalendarYear = 2020;
""")

DatabaseError: Execution failed on sql '
SELECT DISTINCT
    it.IncidentType,
    AVG(CAST(ir.NumberOfIncidents AS DECIMAL)) OVER(PARTITION BY it.IncidentType) AS MeanNumberOfIncidents,
    (SELECT AVG(x.NumberOfIncidents) 
     FROM (SELECT ir2.NumberOfIncidents 
           FROM IncidentRollup ir2 
           INNER JOIN IncidentType it2 ON ir2.IncidentTypeID = it2.IncidentTypeID 
           INNER JOIN Calendar c2 ON ir2.IncidentDate = c2.Date 
           WHERE it2.IncidentType = it.IncidentType AND c2.CalendarQuarter = 2 AND c2.CalendarYear = 2020 
           ORDER BY ir2.NumberOfIncidents 
           LIMIT 2 - (SELECT COUNT(*) 
                      FROM IncidentRollup ir3 
                      INNER JOIN IncidentType it3 ON ir3.IncidentTypeID = it3.IncidentTypeID 
                      INNER JOIN Calendar c3 ON ir3.IncidentDate = c3.Date 
                      WHERE it3.IncidentType = it.IncidentType AND c3.CalendarQuarter = 2 AND c3.CalendarYear = 2020 
                      AND i